In [15]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [27]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()
from langchain_core.globals import set_llm_cache
# from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [41]:
from langchain_core.caches import InMemoryCache

set_llm_cache(InMemoryCache())

In [42]:
from langchain_core.prompts import ChatPromptTemplate

# system_template_old = "Given the following JSON object representing a {category} component in a Revit architectural project, give a short 1-sentence description of the component"

system_template = "You are writing a very short 1-sentence description of a component from a Revit architectural project. The following JSON object contains information about the component. The description is intended as a summary of the component for technical specifications"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{json_object}")]
)

chain = prompt_template | model

In [51]:
import json

filename = 'arch_rac_example_processed'

with open('{}.json'.format(filename), 'r') as file:
    data = json.load(file)

In [52]:
from pprint import pprint as pp

In [57]:
for cat, elems in data.items():

    for elem in elems:
        
        print(elem)
        if "Description Human-Readable" in elem:
            continue
        response = chain.invoke({"category": cat,
                                 "json_object": elem})
        elem["Description Human-Readable"] = response.content
        # break
    break
# print(data["Doors"][0:4])

pp(data)

with open('{}_with_descriptions.json'.format(filename), 'w+') as file2:
        json.dump(data, file2)





{'IfcGUID': '1PDnLIM013wvkZO9Lb4$s7', 'Host Id': 'Wall', 'Family and Type': 'Family Type: 800 x 2100, Family: Single-Flush', 'Materials': ['Door - Frame', 'Door - Panel'], 'Head Height': '2100.0', 'Sill Height': '0.0', 'Area': '2.9649999999999777', 'Volume': '0.10876799999999932', 'Category': 'Doors', 'Level': 'Level(Name=Level 1, Elevation=0)', 'Comments': 'None', 'Description Human-Readable': 'The component is a Single-Flush door measuring 800 x 2100 mm, constructed from Door - Frame and Door - Panel materials, located at Level 1 with a head height of 2100 mm and an area of approximately 2.96 m².'}
{'IfcGUID': '1PDnLIM013wvkZO9Lb4$i6', 'Host Id': 'Wall', 'Family and Type': 'Family Type: Entrance door, Family: Entrance door', 'Materials': ['Glass', 'SH_Aluminum, Anodized Black'], 'Area': '3.8303999999999996', 'Volume': '0.10170816000000032', 'Category': 'Doors', 'Level': 'Level(Name=Level 1, Elevation=0)', 'Comments': 'None'}
{'IfcGUID': '1$p8tACJ938vr1_lKOJJ9g', 'Host Id': 'Wall', 'F